In [1]:
import numpy as np
import tensorflow as tf
import make_datasets as make_ds
import os

print(tf.__version__)

/home/jonas/anaconda3/envs/TF19/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.9.0


In [2]:
num_channels = 3
window_size = 40 # needs to be adjusted for fourier or raw signal

In [3]:
# # for sliding window samples:
# num_vUnits = num_channels*window_size
# num_KCs = 16*num_vUnits
# num_ones = num_KCs*6 #total number of ones in matrix
# p_one = num_ones/(num_vUnits*num_KCs)
# p_WTA = 0.05

# np.random.seed(102)
# rand_binary_matrix = np.random.choice([0, 1], size=(num_vUnits,num_KCs), p=[1-p_one,p_one])

# batch_size = 800

In [4]:
# for fourier samples: 
num_vUnits = int(num_channels*window_size)
num_KCs = 15*num_vUnits
num_ones = num_KCs*6 #total number of ones in matrix
p_one = num_ones/(num_vUnits*num_KCs)
p_WTA = 0.05

np.random.seed(112)
rand_binary_matrix = np.random.choice([0, 1], size=(num_vUnits,num_KCs), p=[1-p_one,p_one])

batch_size = 256

In [5]:
load_path = '/home/jonas/HDD/data/unwindowed/no_emphasis_Fourier_shuffled/'
files = [f for f in os.listdir(load_path)]
print('Num files: ',len(files))

# res_files = []
# load_path = '/home/jonas/data/prepared_data/Fourier_data_ws256_shuffled/'
# for path,dirs,files in os.walk(load_path):
#     for file in files:
#         if 'Kontrolle4' in file or 'Kontrolle5' in file or 'Kontrolle6' in file:
#             res_files.append(file)
# print(len(res_files))
# files = res_files

Num files:  37


In [6]:
random_binary_matrix = tf.constant(rand_binary_matrix.astype(np.float32))

file_name = tf.placeholder(tf.string, shape=[None])
dataset = make_ds.dataset_from_TFRecords(file_name,batch_size,
                                         ['sample','one_hot_label'],['float32','int32'],
                                         shuffle=False,cache=False,parallel_reads=1,num_cores=32)
iterator = dataset.make_initializable_iterator()
batch,label = iterator.get_next()

values,indices = make_ds.make_LSH_values_and_indicies(batch,random_binary_matrix,num_KCs,p_WTA)

with tf.Session() as sess:
    for file in files:
        vals = []
        inds = []
        labels = []
        sess.run(iterator.initializer,feed_dict={file_name:[load_path+file]})
        while True:
            try:
                labels_,inds_,vals_ = sess.run([label,indices,values])
                labels.append(labels_)
                inds.append(inds_)
                vals.append(vals_)
            except tf.errors.OutOfRangeError:
                break
        labels = np.concatenate(labels,axis=0)
        vals = np.concatenate(vals,axis=0)
        inds = np.concatenate(inds,axis=0)
        print('shape WTA values: ', vals.shape)
        print('shape WTA indices: ', inds.shape)
        
        keys_and_raw_features = {'values': vals, 'indices': inds, 'one_hot_label': labels}
        save_name = '/home/jonas/HDD/data/unwindowed/Fourier_no_emphasis_WTA_vals_and_inds_15xKCs_RS112_WTA005_shuffled/' + file
        make_ds.write_to_TFRecord(save_name,keys_and_raw_features)
tf.reset_default_graph()

shape WTA values:  (35310, 90)
shape WTA indices:  (35310, 90)
shape WTA values:  (24960, 90)
shape WTA indices:  (24960, 90)
shape WTA values:  (28680, 90)
shape WTA indices:  (28680, 90)
shape WTA values:  (31710, 90)
shape WTA indices:  (31710, 90)
shape WTA values:  (34140, 90)
shape WTA indices:  (34140, 90)
shape WTA values:  (35490, 90)
shape WTA indices:  (35490, 90)
shape WTA values:  (33000, 90)
shape WTA indices:  (33000, 90)
shape WTA values:  (31260, 90)
shape WTA indices:  (31260, 90)
shape WTA values:  (34890, 90)
shape WTA indices:  (34890, 90)
shape WTA values:  (24270, 90)
shape WTA indices:  (24270, 90)
shape WTA values:  (29820, 90)
shape WTA indices:  (29820, 90)
shape WTA values:  (31980, 90)
shape WTA indices:  (31980, 90)
shape WTA values:  (33240, 90)
shape WTA indices:  (33240, 90)
shape WTA values:  (27210, 90)
shape WTA indices:  (27210, 90)
shape WTA values:  (34290, 90)
shape WTA indices:  (34290, 90)
shape WTA values:  (28140, 90)
shape WTA indices:  (281

In [7]:
# # recover activations for test purposes on a real example
# WTA_reconstructions = []
# batch_size = 100
# dataset = make_ds.dataset_from_TFRecords('/home/jonas/data/prepare_data/WTA_values_and_indices_ws256/Kontrolle3_small_test_file(0).tfrecords',
#                                  batch_size,['values','indices','one_hot_label'],
#                                  ['float32','int32','int32'],shuffle=False,cache=False,num_cores=6)
# iterator = dataset.make_initializable_iterator()
# values,indices,one_hot_labels = iterator.get_next()
# WTA_rec = make_ds.WTA_activations_from_values_and_indices(values,indices,num_KCs,p_WTA)

# with tf.Session() as sess:
#     sess.run(iterator.initializer)
#     while True:
#         try:
#             WTA_reconstructions.append(sess.run(WTA_rec))
#         except tf.errors.OutOfRangeError:
#             break
# tf.reset_default_graph()
# WTA_reconstructions = np.concatenate(WTA_reconstructions,axis=0)
# print('WTA reconstructions: ',np.shape(WTA_reconstructions))
# assert (acts == WTA_reconstructions).all()